# Balancer PID Tuning Utility

### Start Bokeh

In [1]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Range1d
from bokeh.io import push_notebook

output_notebook()

Loading BokehJS ...

In [2]:
from balancer.types import MotionPacket, StatePacket

source = ColumnDataSource(data=dict(timestamp=[0],
                                    pitch=[0], pitch_target=[0],
                                    p=[0], i=[0], d=[0],
                                    velocity=[0], velocity_target=[0],
                                    vp=[0], vi=[0], vd=[0],
                                    yaw=[0], yaw_target=[0],
                                    yp=[0], yi=[0], yd=[0]
                                    ))

orientation = figure(title="Orientation", x_axis_label='Time (ms)', y_axis_label='Pitch (Degrees)', width=800, height=480)
pitch_line = orientation.line('timestamp', 'pitch', source=source, line_width=2, color='blue', alpha=0.8)
pitch_target_line = orientation.line('timestamp', 'pitch_target', source=source, line_width=2, color='red', alpha=0.8)

orientation_pid = figure(title="Orientation PID", x_axis_label='Time (ms)', y_axis_label='Response', width=800, height=480)
p_line = orientation_pid.line('timestamp', 'p', source=source, line_width=2, color='red', alpha=0.8)
i_line = orientation_pid.line('timestamp', 'i', source=source, line_width=2, color='green', alpha=0.8)
d_line = orientation_pid.line('timestamp', 'd', source=source, line_width=2, color='blue', alpha=0.8)

velocity = figure(title="Velocity", x_axis_label='Time (ms)', y_axis_label='Encoder Ticks', width=800, height=480)
velocity_line = velocity.line('timestamp', 'velocity', source=source, line_width=2, color='blue', alpha=0.8)
velocity_target_line = velocity.line('timestamp', 'velocity_target', source=source, line_width=2, color='red', alpha=0.8)
ve_line = velocity.line('timestamp', 'velocity_error', source=source, line_width=2, color='yellow', alpha=0.8)

velocity_pid = figure(title="Velocity PID", x_axis_label='Time (ms)', y_axis_label='Response', width=800, height=480)
vp_line = velocity_pid.line('timestamp', 'vp', source=source, line_width=2, color='red', alpha=0.8)
vi_line = velocity_pid.line('timestamp', 'vi', source=source, line_width=2, color='green', alpha=0.8)
vd_line = velocity_pid.line('timestamp', 'vd', source=source, line_width=2, color='blue', alpha=0.8)

yaw = figure(title="Yaw", x_axis_label='Time (ms)', y_axis_label='Yaw Angle', width=800, height=480)
yaw_line = yaw.line('timestamp', 'yaw', source=source, line_width=2, color='blue', alpha=0.8)
yaw_target_line = yaw.line('timestamp', 'yaw_target', source=source, line_width=2, color='red', alpha=0.8)

yaw_pid = figure(title="Yaw PID", x_axis_label='Time (ms)', y_axis_label='Response', width=800, height=480)
yp_line = yaw_pid.line('timestamp', 'yp', source=source, line_width=2, color='red', alpha=0.8)
yi_line = yaw_pid.line('timestamp', 'yi', source=source, line_width=2, color='green', alpha=0.8)
yd_line = yaw_pid.line('timestamp', 'yd', source=source, line_width=2, color='blue', alpha=0.8)

handle = show(column(orientation, orientation_pid, velocity, velocity_pid, yaw, yaw_pid), notebook_handle=True)

def add_state_packet(state):
    new_data = dict(timestamp=[state.timestamp],
                    pitch=[state.pitch], pitch_target=[state.pitch_target],
                    p=[state.P], i=[state.I], d=[state.D],
                    velocity=[state.velocity], velocity_target=[state.velocity_target],
                    vp=[state.vP], vi=[state.vI], vd=[state.vD],
                    yaw=[state.yaw], yaw_target=[state.yaw_target],
                    yp=[state.yP], yi=[state.yI], yd=[state.yD])
    source.stream(new_data, rollover=1000)
    push_notebook(handle=handle)

In [3]:
import asyncio
import asyncudp
from balancer.types import MotionPacket, PACKET_TYPE_STATE

stop_listening = asyncio.Event()

async def main():
    sock = await asyncudp.create_socket(local_addr=('0.0.0.0', 3000))
    while not stop_listening.is_set():
        data, addr = await sock.recvfrom()
        packet = MotionPacket.from_buffer_copy(data)
        if packet.type == PACKET_TYPE_STATE:
            add_state_packet(packet.data.state)

def stop():
    stop_listening.set()

In [ ]:
await main()